In [1]:
import os
ON_COLAB = "COLAB_GPU" in os.environ

In [2]:
if ON_COLAB:
    !pip install -U pip 2>&1 >/dev/null
    !pip install \
        transformers==4.5.0 \
        fugashi==1.1.0 \
        ipadic==1.0.0 2>&1 \
        torch-lightning==1.2.7 >/dev/null 
    


In [3]:
import random
import glob
from tqdm import tqdm

import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForSequenceClassification

MODEL_NAME = "cl-tohoku/bert-base-japanese-whole-word-masking"

In [13]:
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
bert_sc = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
if ON_COLAB:
    bert_sc = bert_sc.cuda()

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

In [14]:
text_list = [
    "この映画は面白かった。",
    "この映画の最後にはがっかりさせられた。",
    "この映画を見て幸せな気持ちになった。",
]
label_list = [1, 0, 1]

# テキストのトークン化
encoding = tokenizer(text_list, padding="longest", return_tensors="pt")
labels = torch.tensor(label_list)
if ON_COLAB:
    encoding = {k: v.cuda() for k, v in encoding.items()}
    labels = labels.cuda()



In [20]:
# 推論
with torch.no_grad():
    output = bert_sc.forward(**encoding)
scores = output.logits
labels_predicted = scores.argmax(-1)  # スコア最高のラベル
num_correct = (labels_predicted == labels).sum().item()
accurency = num_correct / labels.size(0)


In [16]:
print("スコアのサイズ :", scores.size())
print("予測ラベル     :", labels_predicted)
print("精度           :", accurency)


スコアのサイズ : torch.Size([3, 2])
予測ラベル     : tensor([0, 1, 1])
精度           : 0.3333333333333333


In [22]:
encoding = tokenizer(text_list, padding="longest", return_tensors="pt")
encoding["labels"] = torch.tensor(label_list)
if ON_COLAB:
    encoding = {k: v.cuda() for k, v in encoding.items()}
output = bert_sc(**encoding)
loss = output.loss
print(loss)


tensor(0.7296, grad_fn=<NllLossBackward0>)
